# Load data

# 

- various tasks

[https://github.com/google-research-datasets/Taskmaster/tree/master/TM-1-2019](https://github.com/google-research-datasets/Taskmaster/tree/master/TM-1-2019) 

- Dialogs for booking restaurant

[https://github.com/matthen/dstc](https://github.com/matthen/dstc)

- DailyDialog

[http://yanran.li/dailydialog](http://yanran.li/dailydialog)

- Here dialogs where we can extract plans

[https://github.com/budzianowski/multiwoz](https://github.com/budzianowski/multiwoz)

- Also book restaurants, but here we can extract timeRange and intentions

[https://github.com/sonos/nlu-benchmark/tree/master/2017-06-custom-intent-engines/BookRestaurant](https://github.com/sonos/nlu-benchmark/tree/master/2017-06-custom-intent-engines/BookRestaurant)

- Structured information from texts

[https://github.com/snipsco/snips-nlu](https://github.com/snipsco/snips-nlu)

- Alexa dataset for chat topics

[https://github.com/alexa/Topical-Chat](https://github.com/alexa/Topical-Chat)

- Dialog acts

**[https://web.stanford.edu/~jurafsky/swb1_dialogact_annot.tar.gz](https://web.stanford.edu/~jurafsky/swb1_dialogact_annot.tar.gz)**

[https://aclanthology.org/I17-1099/](https://aclanthology.org/I17-1099/) 

[https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html](https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html) 

[https://github.com/facebookresearch/ParlAI/tree/main/projects/personachat](https://github.com/facebookresearch/ParlAI/tree/main/projects/personachat)

In [1]:
import pandas as pd
import json
import nltk
import spacy
from nltk import word_tokenize
from spacy import displacy
nlp = spacy.load("en_core_web_sm")
nltk.download('universal_tagset')
import tools

[nltk_data] Downloading package universal_tagset to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


## Taskmaster

In [2]:
with open("/workspaces/codespaces-jupyter/data/Taskmaster/TM-1-2019/self-dialogs.json") as f:
    raw_json = f.read()
json_data = json.loads(raw_json)


Я сначала попробую соединять все в одно предложение, потом посомтрим

In [3]:
". ".join([utterance["text"] for utterance in json_data[24]["utterances"]])

"Hello, I need you to order me a car through Lyft. Ok, Where are you located?. I am at the Providence Place Mall in Providence, RI. Ok, and where are you going to?. To Persimmon restaurant on Hope Street. Ok, and is it just you or do you have people traveling with you?. Just me. Ok, is the standard Lyft ok, or would you like the XL?. Standard is fine. How soon would you like to be picked up?. How long will the ride take?. It will be about 8 minutes. Ok, please request the pick up now. Ok, I am requesting pick up for you at Providence Place Mall and stating Persimmon restaurant on Hope St. as your destination. The estimated cost is $7, is this ok?. Yes, please book it with the payment information on file. Ok, your car is booked and should arrive in 4 minutes, I have sent the receipt to your email. Great, what is my eta?. Your estimated time of arrival in 6:23pm. Great, thank you. You are welcome, is there anything else I can help you with?. No that's it for now"

Выделим строчки из json

In [4]:
dataset = []
for idx, dialog in enumerate(json_data):
    dataset.extend([utterance["text"] for utterance in dialog["utterances"]])

In [5]:
print(len(dataset))
print(len(json_data))

169469
7708


Получаем 7708 диалогов и 169469 сообщений. Теперь применим нашу функцию

In [9]:
data = []
for chank_num in range(0, len(dataset)//1000):
    date_sentces = tools.extract_date_roots(dataset[chank_num*1000:(chank_num+1)*1000])
    for i in date_sentces:
        if (i[1] == None):
            data.append([i[0], i[1], i[2], 0])
        else:
            data.append([i[0], i[1], i[2], 1])
    print(f"chunk {chank_num} completed")
data_df = pd.DataFrame(data, columns=["text", "root", "event", "label"])

chunk 0 completed
chunk 1 completed
chunk 2 completed
chunk 3 completed
chunk 4 completed
chunk 5 completed
chunk 6 completed
chunk 7 completed
chunk 8 completed
chunk 9 completed
chunk 10 completed
chunk 11 completed
chunk 12 completed


In [8]:
data = []
for i in date_sentces:
    if (i[1] == None):
        data.append([i[0], i[1], i[2], 0])
    else:
        data.append([i[0], i[1], i[2], 1])
data_df = pd.DataFrame(data, columns=["text", "root", "event", "label"])

,text,root,event,label
0,"Hi , I 'm looking to book a table for Korean f...",None,None,0
1,"Ok , what area are you thinking about ?",None,None,0
2,"Somewhere in Southern NYC , maybe the East Vil...",None,None,0
3,"Ok , great . There 's Thursday Kitchen , it ...",has,has Kitchen Thursday reviews it 's,1
4,That 's great . So I need a table for tonight ...,need,need for 7 table I at pm tonight,1
5,They do n't have any availability for 7 pm .,have,n't pm availability 7 They for have,1
6,What times are available ?,None,None,0
7,5 or 8 .,None,None,0
8,"Yikes , we ca n't do those times .",None,None,0
9,"Ok , do you have a second choice ?",None,None,0
